In [1]:
import pandas as pd
import numpy as np
import seaborn as sns;
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
data = pd.read_csv('C:\\Users\\HARJIT\\OneDrive\\Desktop\\final.csv') 
print(data)

              Startdate           Enddate    NH3     CO   Temp     RH
0      01-01-2012 12:00  01-01-2012 13:00   5.22  16.06  31.58  80.90
1      01-01-2012 13:00  01-01-2012 14:00   5.30  18.35  31.74  79.56
2      01-01-2012 14:00  01-01-2012 15:00   5.11  18.71  31.99  81.53
3      01-01-2012 15:00  01-01-2012 16:00   5.12  18.78  32.33  80.33
4      01-01-2012 16:00  01-01-2012 17:00    NaN    NaN    NaN    NaN
...                 ...               ...    ...    ...    ...    ...
98575  31-03-2023 19:00  31-03-2023 20:00  50.93   1.46    NaN  96.79
98576  31-03-2023 20:00  31-03-2023 21:00  49.48   0.46    NaN  98.50
98577  31-03-2023 21:00  31-03-2023 22:00  49.51   0.64    NaN  98.51
98578  31-03-2023 22:00  31-03-2023 23:00  40.77   0.64    NaN  98.53
98579  31-03-2023 23:00  01-04-2023 00:00  42.16   0.50    NaN  98.55

[98580 rows x 6 columns]


In [3]:
data['NH3'] = data['NH3'].ffill()
data['NH3'] = data['NH3'].bfill()


data['CO'] = data['CO'].ffill()
data['CO'] = data['CO'].bfill()

data['Temp'] = data['Temp'].ffill()
data['Temp'] = data['Temp'].bfill()

data['RH'] = data['RH'].ffill()
data['RH'] = data['RH'].bfill()

In [4]:
print(data)

              Startdate           Enddate    NH3     CO   Temp     RH
0      01-01-2012 12:00  01-01-2012 13:00   5.22  16.06  31.58  80.90
1      01-01-2012 13:00  01-01-2012 14:00   5.30  18.35  31.74  79.56
2      01-01-2012 14:00  01-01-2012 15:00   5.11  18.71  31.99  81.53
3      01-01-2012 15:00  01-01-2012 16:00   5.12  18.78  32.33  80.33
4      01-01-2012 16:00  01-01-2012 17:00   5.12  18.78  32.33  80.33
...                 ...               ...    ...    ...    ...    ...
98575  31-03-2023 19:00  31-03-2023 20:00  50.93   1.46  38.74  96.79
98576  31-03-2023 20:00  31-03-2023 21:00  49.48   0.46  38.74  98.50
98577  31-03-2023 21:00  31-03-2023 22:00  49.51   0.64  38.74  98.51
98578  31-03-2023 22:00  31-03-2023 23:00  40.77   0.64  38.74  98.53
98579  31-03-2023 23:00  01-04-2023 00:00  42.16   0.50  38.74  98.55

[98580 rows x 6 columns]


In [5]:
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x > 1 and x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x > 2 and x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x > 10 and x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x > 17 and x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

data["ci"] = data["CO"].astype(float).apply(lambda x: get_CO_subindex(x))

def get_NH3_subindex(x):
    if x <= 200:
        return x * 50 / 200
    elif x > 200 and x <= 400:
        return 50 + (x - 200) * 50 / 200
    elif x > 400 and x <= 800:
        return 100 + (x - 400) * 100 / 400
    elif x > 800 and x <= 1200:
        return 200 + (x - 800) * 100 / 400
    elif x > 1200 and x <= 1800:
        return 300 + (x - 1200) * 100 / 600
    elif x > 1800:
        return 400 + (x - 1800) * 100 / 600
    else:
        return 0

data["ni"] = data["NH3"].astype(float).apply(lambda x: get_NH3_subindex(x))



In [6]:
def calculate_aqi(ni,ci):
    aqi=0
  
    if(ni>ci):
     aqi=ni
    if(ci>ni):
      aqi=ci
  
    return aqi
data['AQI']=data.apply(lambda x:calculate_aqi(x['ni'],x['ci']),axis=1)
data.head()

,Startdate,Enddate,NH3,CO,Temp,RH,ci,ni,AQI
0,01-01-2012 12:00,01-01-2012 13:00,5.22,16.06,31.58,80.90,286.571429,1.3050,286.571429
1,01-01-2012 13:00,01-01-2012 14:00,5.30,18.35,31.74,79.56,307.941176,1.3250,307.941176
2,01-01-2012 14:00,01-01-2012 15:00,5.11,18.71,31.99,81.53,310.058824,1.2775,310.058824
3,01-01-2012 15:00,01-01-2012 16:00,5.12,18.78,32.33,80.33,310.470588,1.2800,310.470588
4,01-01-2012 16:00,01-01-2012 17:00,5.12,18.78,32.33,80.33,310.470588,1.2800,310.470588


In [7]:
data.isnull().sum()

Startdate    0
Enddate      0
NH3          0
CO           0
Temp         0
RH           0
ci           0
ni           0
AQI          0
dtype: int64

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

data['AQI'] = pd.to_numeric(data['AQI'], errors='coerce')

def get_aqi_level(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Satisfactory"
    elif aqi <= 200:
        return "Moderate"
    elif aqi <= 300:
        return "Poor"
    elif aqi <= 400:
        return "Very Poor"
    else:
        return "Severe"

data['AQI_Level'] = data['AQI'].apply(get_aqi_level)

X = data[['ci', 'ni']]  
y = data['AQI_Level']  

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)


param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 2]
}

# Initialize Random Forest Regressor
model = RandomForestClassifier(random_state=0)

# Perform Grid Search to find the best parameters
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best model from GridSearchCV
best_model = grid_search.best_estimator_

# Get CO and NH3 values from user input
CO_subindex = float(input("Enter the CO subindex: "))
NH3_subindex = float(input("Enter the NH3 subindex: "))

# Calculate CO and NH3 subindices using provided functions
ci = get_CO_subindex(CO_subindex)
ni = get_NH3_subindex(NH3_subindex)

input_aqi = calculate_aqi(ni,ci)
print("Input AQI:", input_aqi)

# Predict AQI using the trained model and calculated CO and NH3 subindices
predicted_aqi_label = best_model.predict([[ci, ni]])
print("Predicted AQI Level:", predicted_aqi_label)

y_val_pred = best_model.predict(X_val)
accuracy_val = accuracy_score(y_val, y_val_pred)
print("Validation Set Accuracy:", accuracy_val)

# Evaluate model accuracy on test set
y_test_pred = best_model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_test_pred)
print("Test Set Accuracy:", accuracy_test)


Fitting 3 folds for each of 16 candidates, totalling 48 fits
Enter the CO subindex: 2
Enter the NH3 subindex: 3
Input AQI: 100.0
Predicted AQI Level: ['Satisfactory']


C:\Users\HARJIT\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Validation Set Accuracy: 0.9998647460607291
Test Set Accuracy: 0.9998647460607291


In [9]:
!pip install firebase-admin


In [10]:
import firebase_admin
from firebase_admin import credentials, db, firestore
from firebase_admin import _apps as firebase_apps

In [12]:
import firebase_admin
from firebase_admin import credentials, db
import random
import time 


cred_path = "C:\\Users\\HARJIT\\Downloads\\iot-and-esp-32-firebase-adminsdk-j6hfv-0c533f4313.json"
database_url = 'https://iot-and-esp-32-default-rtdb.asia-southeast1.firebasedatabase.app/'

cred = credentials.Certificate(cred_path)

firebase_admin.initialize_app(cred, {
    'databaseURL': database_url
})
pollutants_ref = db.reference('pollutant_values')
def listen_for_realtime_updates():
    def on_new_pollutant_data(event):
        # Check if the event is a 'put' and the path contains a '/'
        if event.event_type == 'put' and '/' in event.path:
            parts = event.path.strip('/').split('/')
            if len(parts) == 2:
                child_key, _ = parts
                try:
                    value = float(event.data)
                except ValueError:
                    # If there's an error in conversion, print it and return early
                    print(f"Invalid data for AQI: {event.data}")
                    return
                
      
                aqi_level_string = get_aqi_level(value)
                pollutants_ref.child(child_key).update({
                    'AQI Level String': aqi_level_string
                })
                print(f"AQI Level updated for {child_key}")

    # Listen to the database reference
    pollutants_ref.listen(on_new_pollutant_data)

listen_for_realtime_updates()


AQI Level updated for 272711
AQI Level updated for 272711
AQI Level updated for 272711
AQI Level updated for 282712
AQI Level updated for 282712
AQI Level updated for 282712
AQI Level updated for 292713
AQI Level updated for 292713
AQI Level updated for 292713
AQI Level updated for 302714
AQI Level updated for 302714
AQI Level updated for 302714
AQI Level updated for 312715
AQI Level updated for 312715
AQI Level updated for 312715
AQI Level updated for 322716
AQI Level updated for 322716
AQI Level updated for 322716
AQI Level updated for 332717
AQI Level updated for 332717
AQI Level updated for 332717
AQI Level updated for 342718
AQI Level updated for 342718
AQI Level updated for 342718
AQI Level updated for 352719
AQI Level updated for 352719
AQI Level updated for 352719
AQI Level updated for 362720
AQI Level updated for 362720
AQI Level updated for 362720
AQI Level updated for 372721
AQI Level updated for 372721
AQI Level updated for 372721
AQI Level updated for 382722
AQI Level upda